<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/cnn/cifar_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

if torch.backends.mps.is_available():
    my_device = torch.device('mps')
elif torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 1. Data loading and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing for each RGB channel
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

# 2. Define the mynet model
class MyNet(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        super().__init__()
        self.seq_model = nn.Sequential(
            nn.Linear(input_size, hidden_size, bias=True),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size, bias=True),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size, bias=True),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size, bias=True),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.view(x.size(0), -1)  # Reshape the input here
        return self.seq_model(x)

mynet = MyNet(input_size=32*32*3, hidden_size=128, output_size=10)


100%|██████████| 170498071/170498071 [00:05<00:00, 28931684.78it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [3]:
correct_count = 0
with torch.no_grad():
    for data, targets in val_loader:
        scores = mynet(data)
        predicted = scores.argmax(dim=1)
        correct_count += predicted.eq(targets).sum().item()

    val_accuracy = 100. * correct_count / len(val_loader.dataset)

print(f"First Validation Accuracy: {val_accuracy:.2f}%")


First Validation Accuracy: 10.09%


In [ ]:
# 3. Initialize loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(mynet.parameters(), lr=0.001)

# 4. Train the model
mynet.to(my_device)
num_epochs = 100
for epoch in range(num_epochs):
    mynet.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(my_device), targets.to(my_device)
        scores = mynet(data)
        loss = loss_fn(scores, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    mynet.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(my_device), targets.to(my_device)
            scores = mynet(data)
            loss = loss_fn(scores, targets)
            val_loss += loss.item() * data.size(0)

            predicted = scores.argmax(dim=1)
            correct += predicted.eq(targets).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = 100. * correct / len(val_loader.dataset)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch [1/100], Training Loss: 1.4732, Validation Loss: 1.5931, Validation Accuracy: 44.61%
Epoch [2/100], Training Loss: 1.4671, Validation Loss: 1.5102, Validation Accuracy: 47.38%
Epoch [3/100], Training Loss: 1.5537, Validation Loss: 1.4609, Validation Accuracy: 48.14%
Epoch [4/100], Training Loss: 1.4527, Validation Loss: 1.4225, Validation Accuracy: 49.55%
Epoch [5/100], Training Loss: 1.3204, Validation Loss: 1.4116, Validation Accuracy: 49.99%
Epoch [6/100], Training Loss: 1.2641, Validation Loss: 1.4001, Validation Accuracy: 51.05%
Epoch [7/100], Training Loss: 1.1481, Validation Loss: 1.3734, Validation Accuracy: 52.09%
Epoch [8/100], Training Loss: 1.4672, Validation Loss: 1.3686, Validation Accuracy: 52.55%
